In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [3]:
#@ Loading the required libraries
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [4]:
#@ Loading the required dataset
data_train = pd.read_csv('/kaggle/input/fake-news/train.csv')
data_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
#@ Checking shape of dataset
data_train.shape

(20800, 5)

In [6]:
#@ Checking if there is any missing values
data_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#@ Let's drop the null values
data = data_train.dropna()

In [8]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
#@ Let's get Independent and Dependent features
X = data.drop('label', axis=1)
y = data['label']

In [10]:
#@ Checking shape after extracting features
print('The shape of X is:',X.shape)
print('The shape of y is:',y.shape)

The shape of X is: (18285, 4)
The shape of y is: (18285,)


In [11]:
#@ Defining vocabulary size
voc_size=5000

**One Hot Representation**

In [12]:
#@ Copying the X values as messages
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [13]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [14]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
#@ Let's apply the preprocessing in the dataset
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-z0-9]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi c

In [16]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [17]:
data['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [18]:
#@ Now, let's apply OHE in the corpus
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3287, 1207, 1522, 2069, 3283, 4979, 4968, 4977, 4455, 1428],
 [32, 2519, 3282, 3350, 3493, 3311, 2404],
 [4004, 31, 3840, 2244],
 [704, 1067, 2735, 3455, 340, 4317, 1144],
 [2780, 3493, 1268, 1281, 3949, 712, 3493, 2587, 2702, 702],
 [3856,
  3128,
  378,
  2562,
  1600,
  948,
  1233,
  4603,
  4435,
  2965,
  2435,
  488,
  620,
  3007,
  2404],
 [1086, 2971, 106, 3241, 1186, 484, 734, 663, 421, 3467, 4922],
 [3163, 3262, 3598, 674, 687, 3720, 948, 2242, 421, 3467, 4922],
 [4420, 3106, 3602, 1222, 2338, 4030, 1463, 4133, 948, 4997],
 [2493, 3984, 2171, 2768, 874, 4279, 1637, 1534],
 [205, 1045, 3224, 3992, 2298, 3857, 3173, 4024, 4540, 383, 4133],
 [340, 784, 3283, 4030, 948, 687],
 [441, 4458, 2185, 1286, 323, 2990, 3340, 3916, 1519],
 [161, 612, 3631, 598, 4841, 942, 3422, 421, 3467, 4922],
 [108, 21, 26, 4790, 3910, 421, 3467, 4922],
 [2248, 4382, 3090, 757, 1362, 4818, 1719, 1062, 1355, 3746, 808, 1277],
 [917, 4227, 2519],
 [2110, 2089, 272, 1684, 948, 1284, 4624, 2404],
 [402

In [19]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [20]:
onehot_repr[0]

[3287, 1207, 1522, 2069, 3283, 4979, 4968, 4977, 4455, 1428]

**Embedding representation**

In [21]:
#@ Let's apply post padding in each sentences
sent_length=20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
embedded_docs

array([[3287, 1207, 1522, ...,    0,    0,    0],
       [  32, 2519, 3282, ...,    0,    0,    0],
       [4004,   31, 3840, ...,    0,    0,    0],
       ...,
       [2350, 3576, 2878, ...,    0,    0,    0],
       [1267,  687, 3029, ...,    0,    0,    0],
       [1119, 4413, 1059, ...,    0,    0,    0]], dtype=int32)

In [22]:
embedded_docs[0]

array([3287, 1207, 1522, 2069, 3283, 4979, 4968, 4977, 4455, 1428,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [23]:
#@ Now, let's create the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs), y.shape

(18285, (18285,))

In [25]:
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [27]:
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 36ms/step - loss: 0.3218 - accuracy: 0.8406 - val_loss: 0.2029 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1340 - accuracy: 0.9500 - val_loss: 0.2026 - val_accuracy: 0.9142
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0881 - accuracy: 0.9691 - val_loss: 0.2516 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0651 - accuracy: 0.9786 - val_loss: 0.2679 - val_accuracy: 0.9118
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0463 - accuracy: 0.9854 - val_loss: 0.2803 - val_accuracy: 0.9110
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0362 - accuracy: 0.9883 - val_loss: 0.2929 - val_accuracy: 0.9051
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0241 - accuracy: 0.9935 - val_loss: 0.3848 - val_accuracy: 0.909

- The model seems to perform well on the training data, with the training accuracy improving from 0.84 in the first epoch to 0.9961 in the ninth epoch. However, the validation accuracy does not improve as much, hovering around 0.91 throughout training.
- This suggests that the model is overfitting to the training data and not generalizing well to new data. The increasing validation loss in later epochs also supports this hypothesis. The model may benefit from additional regularization techniques, such as dropout or weight decay, to prevent overfitting.

In [28]:
#@ Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
#@ Applying train/test/split
X_final = np.array(embedded_docs)
y_final=np.array(y)
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 39ms/step - loss: 0.3128 - accuracy: 0.8450 - val_loss: 0.2079 - val_accuracy: 0.9079
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1472 - accuracy: 0.9443 - val_loss: 0.1921 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1026 - accuracy: 0.9652 - val_loss: 0.2195 - val_accuracy: 0.9165
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0755 - accuracy: 0.9742 - val_loss: 0.2736 - val_accuracy: 0.9153
Epoch 5/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0601 - accuracy: 0.9808 - val_loss: 0.3230 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0481 - accuracy: 0.9849 - val_loss: 0.3239 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0382 - accuracy: 0.9879 - val_loss: 0.3521 - val_accuracy: 0.910

- The result after applying dropout technique shows a slightly better validation accuracy than the previous result. This means that dropout helped reduce overfitting and improve the generalization of the model.
- The training accuracy decreased slightly, which is expected because dropout randomly drops out some connections during training, making it harder for the model to fit the training data perfectly. However, this trade-off is necessary to achieve better performance on unseen data.

In [30]:
# Define early stopping criteria
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

# Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))

In [31]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[earlystop])

Epoch 1/100
192/192 [==============================] - 10s 38ms/step - loss: 0.3128 - accuracy: 0.8460 - val_loss: 0.2082 - val_accuracy: 0.9115
Epoch 2/100
192/192 [==============================] - 7s 35ms/step - loss: 0.1421 - accuracy: 0.9436 - val_loss: 0.2018 - val_accuracy: 0.9218
Epoch 3/100
192/192 [==============================] - 7s 35ms/step - loss: 0.0988 - accuracy: 0.9651 - val_loss: 0.1997 - val_accuracy: 0.9160
Epoch 4/100
192/192 [==============================] - 7s 34ms/step - loss: 0.0776 - accuracy: 0.9736 - val_loss: 0.2256 - val_accuracy: 0.9181
Epoch 5/100
192/192 [==============================] - 7s 34ms/step - loss: 0.0563 - accuracy: 0.9815 - val_loss: 0.3205 - val_accuracy: 0.9138
Epoch 6/100
192/192 [==============================] - 7s 35ms/step - loss: 0.0401 - accuracy: 0.9865 - val_loss: 0.3119 - val_accuracy: 0.9110
Epoch 6: early stopping


- The use of callbacks has stopped training early on the 6th epoch since the validation loss did not improve in the last 3 epochs. This is a good sign that our model is not overfitting and has reached a good level of accuracy. The final validation accuracy achieved by the model is 91.10%.